# Use the Model View Projection

# Study Weights of the Model Out of The Box

In [1]:
import numpy as np
import tensorflow as tf
import torch
from transformers import AutoTokenizer, TFCamembertForMaskedLM
from bertviz import model_view
import matplotlib.pyplot as plt
import IPython.core.display as display

In [2]:
### instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("jplu/tf-camembert-base", do_lower_case=True)

In [3]:
# ### path to weights
# checkpointPath = "../project_PunctuatorBERTTensorFlow2/Models/20200512_083058/cp-010.ckpt"

In [4]:
### punctuation encoder
punctuation_enc = {
    'O': 0,
    'PERIOD': 1,
}

In [5]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min',
    jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [6]:
### hyperparameters
vocab_size = 32005
sequenceSize = 32

In [7]:
### get model
bertInp = tf.keras.Input(shape=(sequenceSize), dtype='int32', name='bertInp')
bertOut = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base", output_attentions=True)(bertInp)
x = tf.keras.layers.Reshape((sequenceSize*vocab_size,))(bertOut[0])
denseOut = tf.keras.layers.Dense(2)(x)

model = tf.keras.Model(
    inputs = [bertInp],
    outputs = [bertOut, denseOut],
)

In [8]:
# ### load the weights
# model.load_weights(checkpointPath)

### Input Sentence

In [9]:
# sfr bonjour bienvenue au service client <b> oui bonjour monsieur <b> je vous appelle parce que j'ai recu un colis dans un relais <b>

In [10]:
# string = "sfr bonjour bienvenue au service client oui bonjour monsieur je vous appelle parce que j'ai recu un colis dans un relais"
string = "sfr bonjour bienvenue au service client oui bonjour monsieur je vous appelle parce que j'ai recu un colis dans un relais sfr service client bonjour william"
# string = "vous avez une taille de facture totalement a bouffer sfr bonjour a votre ecoute oui bonjour monsieur parce que en fait je n'arrive pas a aller sur mon"
# string = "en fait dans la d'internet que je je suis abonnee a sfr et voila j'ai du depasser en fait ben tout de suite un"

tokens = tokenizer.tokenize(string)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)

['▁s', 'fr', '▁bonjour', '▁bienvenue', '▁au', '▁service', '▁client', '▁oui', '▁bonjour', '▁monsieur', '▁je', '▁vous', '▁appelle', '▁parce', '▁que', '▁j', "'", 'ai', '▁re', 'cu', '▁un', '▁colis', '▁dans', '▁un', '▁relais', '▁s', 'fr', '▁service', '▁client', '▁bonjour', '▁will', 'iam']


In [11]:
print(len(ids))
sentenceSize = len(ids)

32


In [12]:
print(ids)

[52, 427, 5061, 5819, 36, 366, 1065, 773, 5061, 5615, 50, 39, 2668, 398, 27, 76, 11, 73, 343, 3172, 23, 5990, 29, 23, 6542, 52, 427, 366, 1065, 5061, 14233, 10838]


In [13]:
### pad the sequence
foo = []
foo.append(ids)
test = tf.keras.preprocessing.sequence.pad_sequences(foo, maxlen=32, padding='post')
print(test)

[[   52   427  5061  5819    36   366  1065   773  5061  5615    50    39
   2668   398    27    76    11    73   343  3172    23  5990    29    23
   6542    52   427   366  1065  5061 14233 10838]]


In [14]:
modelOut = model.predict(test)
bertOutput = modelOut[0]

In [15]:
# get the attention weights
attLayers = bertOutput[1]

In [16]:
# convert weights from np array to torch tensors
attention = []
for i in range(len(attLayers)):
    foo = torch.from_numpy(attLayers[i])
    attention.append(foo)
attention=tuple(attention)

In [17]:
sequence = tokenizer.convert_ids_to_tokens(test[0])
print(sequence)

['▁s', 'fr', '▁bonjour', '▁bienvenue', '▁au', '▁service', '▁client', '▁oui', '▁bonjour', '▁monsieur', '▁je', '▁vous', '▁appelle', '▁parce', '▁que', '▁j', "'", 'ai', '▁re', 'cu', '▁un', '▁colis', '▁dans', '▁un', '▁relais', '▁s', 'fr', '▁service', '▁client', '▁bonjour', '▁will', 'iam']


In [18]:
# %%capture --no-stderr --no-stdout output

# ### head_view(attention, sequence)

# ### model_view(attention, tokens, sentence_b_start)

img = model_view(attention, sequence)

<IPython.core.display.Javascript object>

In [19]:
type(img)

IPython.core.display.Javascript

In [20]:
display.display(img)

<IPython.core.display.Javascript object>